In [16]:
def classify_color_by_hue(hue_value):
    if 345 <= hue_value <= 360 or 0 <= hue_value < 15:
        return "Red"
    elif 15 <= hue_value < 30:
        return "Red-Orange"
    elif 30 <= hue_value < 45:
        return "Orange"
    elif 45 <= hue_value < 60:
        return "Yellow-Orange"
    elif 60 <= hue_value < 75:
        return "Yellow"
    elif 75 <= hue_value < 90:
        return "Yellow-Green"
    elif 90 <= hue_value < 105:
        return "Green"
    elif 105 <= hue_value < 120:
        return "Blue-Green"
    elif 120 <= hue_value < 135:
        return "Cyan"
    elif 135 <= hue_value < 150:
        return "Blue-Cyan"
    elif 150 <= hue_value < 165:
        return "Blue"
    elif 165 <= hue_value < 180:
        return "Blue-Purple"
    elif 180 <= hue_value < 195:
        return "Purple"
    elif 195 <= hue_value < 210:
        return "Purple-Magenta"
    elif 210 <= hue_value < 225:
        return "Magenta"
    elif 225 <= hue_value < 240:
        return "Red-Magenta"
    elif 240 <= hue_value < 255:
        return "Pink"
    elif 255 <= hue_value < 270:
        return "Light Pink"
    elif 270 <= hue_value < 285:
        return "Very Light Pink"
    elif 285 <= hue_value < 300:
        return "Pale Pink"
    elif 300 <= hue_value < 315:
        return "Grayish Pink"
    elif 315 <= hue_value < 330:
        return "Grayish Purple"
    else:
        return "Undefined"


def rgb_to_hue(rgb):
    # Normalize RGB values to the range [0, 1]
    r, g, b = [x / 255.0 for x in rgb]

    max_val = max(r, g, b)
    min_val = min(r, g, b)

    # Calculate hue
    if max_val == min_val:
        h = 0.0
    elif max_val == r:
        h = (g - b) / (max_val - min_val)
    elif max_val == g:
        h = 2.0 + (b - r) / (max_val - min_val)
    elif max_val == b:
        h = 4.0 + (r - g) / (max_val - min_val)

    h *= 60.0
    if h < 0.0:
        h += 360.0

    return h

In [17]:
import cv2
import numpy as np
import requests
from collections import Counter


def image2color(url):
    image_url = url

    # Download the image
    response = requests.get(image_url)
    image = cv2.imdecode(np.frombuffer(response.content, np.uint8), cv2.IMREAD_COLOR)

    # Get image dimensions
    height, width, _ = image.shape

    # Define crop percentages
    top_crop_percent = 20
    bottom_crop_percent = 20
    left_crop_percent = 25
    right_crop_percent = 25

    # Calculate crop sizes
    top_crop = int(height * (top_crop_percent / 100))
    bottom_crop = int(height * (bottom_crop_percent / 100))
    left_crop = int(width * (left_crop_percent / 100))
    right_crop = int(width * (right_crop_percent / 100))

    # Perform cropping
    cropped_image = image[
        top_crop : height - bottom_crop, left_crop : width - right_crop
    ]
    cropped_image = cv2.cvtColor(cropped_image, cv2.COLOR_BGR2RGB)
    

    h_values = np.array(
        list(map(lambda x: rgb_to_hue(x), cropped_image.reshape(-1, 3)))
    )
    most_common_colors = Counter(
        list(map(lambda x: classify_color_by_hue(x), h_values))
    ).most_common(4)

 
   
    val=0
    for color in most_common_colors:
        val+=color[1]
    
    result = list(map(lambda x: {x[0]: x[1] / val},most_common_colors))
    final_dict = {}
    for color in result:
        final_dict[list(color.keys())[0]] = list(color.values())[0]
    
    return final_dict

In [18]:
from pymongo import MongoClient

CONNECTION_STRING = "mongodb://localhost:27017"
client = MongoClient(CONNECTION_STRING)
db = client["florence"]
collection = db["trueproducts"]



In [19]:
products = collection.find({})
images = list(map(lambda x: {"id":x["id"],"img_url": x["images"][0]}, products))
img=images[0:3]
img

[{'id': '6021b5d0-c12c-4c82-b515-b5ebc04367fe',
  'img_url': 'https://www.fnp.com/images/pr/x/v20200611181918/100-red-yellow-roses-premium-arrangement_1.jpg'},
 {'id': 'ddd1a4d5-e1d0-4608-abed-d29cf0f379d5',
  'img_url': 'https://www.fnp.com/images/pr/x/v20200611181916/100-red-roses-premium-arrangement_1.jpg'},
 {'id': '559f2849-4e34-4afd-8303-e1acad51e247',
  'img_url': 'https://www.fnp.com/images/pr/x/v20200128201147/1000-red-roses-grand-teddy-bear_1.jpg'}]

In [20]:
def groupData(data, nGroups):
    data = [i for i in data]
    num = int(len(data))
    groups = []
    o = range(0, num, int(num / nGroups))
    for i in o:
        groups.append(data[i : i + int(num / nGroups)])
    return groups

In [21]:
color_coll = db["colors"]

In [22]:
import threading


def threadFunc(images):
    for x in images:
        color_coll.insert_one({"id": x["id"], "color": image2color(x["img_url"])})


threads = []
for group in groupData(images, 2):
    x = threading.Thread(
        target=threadFunc,
        args=(group,),
    )
    threads.append(x)
    x.start()

for thread in threads:
    thread.join()